In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from statsmodels.tsa.seasonal import seasonal_decompose
import math

from sklearn.metrics import mean_absolute_error, mean_squared_error
from datetime import datetime, date

import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv('data/Aquifer_Petrignano.csv')

# Drop data before 2009
df = df[df.Rainfall_Bastia_Umbra.notna()].reset_index(drop=True)

# Drop one of the target columns, so we can focus on only one target
df = df.drop(['Depth_to_Groundwater_P24', 'Temperature_Petrignano'],
            axis=1)

# Simplify column names
df.columns = ['Date', 'Rainfall', 'Depth_to_Groundwater',
             'Temperature', 'Drainage_Volume', 'River_Hydrometry']

targets = ['Depth_to_Groundwater']
features = [feature for feature in df.columns if feature not in targets]

df.head()

In [ ]:
df['Date'] = pd.to_datetime(df.Date, format = '%d/%m/%Y')
df.head().style.set_properties(subset=['Date'], **{
    'background-color': 'dodgerblue'
})

In [ ]:
fig = make_subplots(rows=5, cols=1,
                    subplot_titles=('Feature: Rainfall',
                                    'Feature: Temperature',
                                    'Feature: Volume',
                                    'Feature: Hydrometry',
                                    'Feature: Depth to Groundwater'))

fig.add_trace(
    go.Scatter(x=df.Date, y=df.Rainfall.fillna(np.inf), marker_color='red'),
    row = 1,
    col = 1,
)

fig.add_trace(
    go.Scatter(x=df.Date, y=df.Temperature.fillna(np.inf)),
    row=2, 
    col=1,
)

fig.add_trace(
    go.Scatter(x=df.Date, y=df.Drainage_Volume.fillna(np.inf)),
    row=3,
    col=1
)

fig.add_trace(
    go.Scatter(x=df.Date, y=df.River_Hydrometry.fillna(np.inf)),
    row=4,
    col=1
)

fig.add_trace(
    go.Scatter(x=df.Date, y=df.Depth_to_Groundwater.fillna(np.inf)),
    row=5,
    col=1
)

# Update yaxis properties
fig.update_yaxes(title_text='Rainfall', row=1, col=1)
fig.update_yaxes(title_text='Temperature', row=2, col=1)
fig.update_yaxes(title_text='Volume', row=3, col=1)
fig.update_yaxes(title_text='Hydrometry', row=4, col=1)
fig.update_yaxes(title_text='Depth to Groundwater', row=5, col=1)

# Update xaxis properties
fig.update_xaxes(title_text='Date', row=5, col=1)

fig.update_layout(height=1000, width=1000, showlegend=False)

In [ ]:
# Sort values by timestamp 
df = df.sort_values(by='Date')

# Check time intervals
df['Time_Interval'] = df.Date - df.Date.shift(1)

df[['Date', 'Time_Interval']].head()

# Handling Missing Values

In [ ]:
fig = make_subplots(rows = 2, cols = 1)

old = df.River_Hydrometry.copy()
df['River_Hydrometry'] = np.where((df.River_Hydrometry == 0),
                                 np.nan,
                                 df.River_Hydrometry)

fig.add_trace(
    go.Scatter(
        x = df.Date,
        y = old.fillna(np.inf),
        name = 'Original'
    ),
    row=1,
    col=1
)

fig.add_trace(
    go.Scatter(
        x = df.Date,
        y = df.River_Hydrometry.fillna(np.inf),
        name = 'Modiefied'
    ),
    row=1,
    col=1
)

fig.update_yaxes(title_text='Hydrometry', row=1, col=1)
 
old = df.Drainage_Volume.copy()
df['Drainage_Volume'] = np.where((df.Drainage_Volume == 0),
                              np.nan,
                              df.Drainage_Volume)

fig.add_trace(
    go.Scatter(
        x = df.Date,
        y = old.fillna(np.inf),
        name = 'Original'
    ),
    row = 2,
    col = 1
)

fig.add_trace(
    go.Scatter(
        x = df.Date,
        y = df.Drainage_Volume.fillna(np.inf),
        name = 'Modified'
    ),
    row = 2,
    col = 1
)

fig.update_yaxes(title_text='Volume', row=2, col=1)
fig.update_xaxes(title_text='Date', row=2, col=1)

fig.update_layout(width=1000, height=600)

We gona use imputation technique to fill missing values 

In [ ]:
df['Drainage_Volume'] = df['Drainage_Volume'].interpolate()
df['River_Hydrometry'] = df['River_Hydrometry'].interpolate()
df['Depth_to_Groundwater'] = df['Depth_to_Groundwater'].interpolate()

# Resampling

We will do some downsampling with the resample() function

In [ ]:
fig = make_subplots(rows=4, cols=2,
                    subplot_titles=(
                        'Daily Rainfall (Acc.)',
                        'Daily Temperature (Acc.)',
                        'Weekly Rainfall (Acc.)',
                        'Weekly Temperature (Acc.)',
                        'Monthly Rainfall (Acc.)',
                        'Monthly Temperature (Acc.)',
                        'Annual Rainfall (Acc.)',
                        'Annual Temperature (Acc.)'
                    ))

resampled_df = df[['Date', 'Rainfall']].resample('D', on='Date').sum().reset_index(drop=False)
fig.add_trace(
    go.Scatter(
        x = resampled_df.Date,
        y = resampled_df.Rainfall
    ),
    row = 1,
    col = 1
)
fig.update_yaxes(title_text='Rainfall', row=1, col=1)


resampled_df = df[['Date', 'Rainfall']].resample('7D', on='Date').sum().reset_index(drop=False)
fig.add_trace(
    go.Scatter(
        x = resampled_df.Date,
        y = resampled_df.Rainfall
    ),
    row = 2,
    col = 1
)
fig.update_yaxes(title_text='Rainfall', row=2, col=1)

resampled_df = df[['Date', 'Rainfall']].resample('M', on='Date').sum().reset_index(drop=False)
fig.add_trace(
    go.Scatter(
        x = resampled_df.Date,
        y = resampled_df.Rainfall
    ),
    row = 3,
    col = 1
)
fig.update_yaxes(title_text='Rainfall', row=3, col=1)

resampled_df = df[['Date', 'Rainfall']].resample('Y', on='Date').sum().reset_index(drop=False)
fig.add_trace(
    go.Scatter(
        x = resampled_df.Date,
        y = resampled_df.Rainfall
    ),
    row = 4,
    col = 1
)
fig.update_yaxes(title_text='Rainfall', row=4, col=1)
fig.update_xaxes(title_text='Date', row=4, col=1)

resampled_df = df[['Date', 'Temperature']].resample('D', on='Date').sum().reset_index(drop=False)
fig.add_trace(
    go.Scatter(
        x = resampled_df.Date,
        y = resampled_df.Temperature
    ),
    row = 1,
    col = 2
)
fig.update_yaxes(title_text='Temperature', row=1, col=2)

resampled_df = df[['Date', 'Temperature']].resample('7D', on='Date').sum().reset_index(drop=False)
fig.add_trace(
    go.Scatter(
        x = resampled_df.Date,
        y = resampled_df.Temperature
    ),
    row = 2,
    col = 2
)
fig.update_yaxes(title_text='Temperature', row=2, col=2)

resampled_df = df[['Date', 'Temperature']].resample('M', on='Date').sum().reset_index(drop=False)
fig.add_trace(
    go.Scatter(
        x = resampled_df.Date,
        y = resampled_df.Temperature
    ),
    row = 3,
    col = 2
)
fig.update_yaxes(title_text='Temperature', row=3, col=2)

resampled_df = df[['Date', 'Temperature']].resample('Y', on='Date').sum().reset_index(drop=False)
fig.add_trace(
    go.Scatter(
        x = resampled_df.Date,
        y = resampled_df.Temperature
    ),
    row = 4,
    col = 2
)
fig.update_yaxes(title_text='Temperature', row=4, col=2)
fig.update_xaxes(title_text='Date', row=4, col=2)

fig.update_layout(height=1000, width=2000)

There is no necessity to look at the daily data. Considering weekly data seems to be sufficient as well. 

In [ ]:
df_downsampled = df[[
    'Date',
    'Depth_to_Groundwater',
    'Temperature',
    'Drainage_Volume',
    'River_Hydrometry'
]].resample('7D', on='Date').mean().reset_index(drop=False)

df_downsampled['Rainfall'] = df[[
    'Date',
    'Rainfall'
]].resample('7D', on='Date').sum().reset_index(drop=False)[['Rainfall']]

df = df_downsampled

# Stationarity

Plot time series and check for trends or seasonality

In [ ]:
fig = make_subplots(rows = 3, cols = 2, subplot_titles=(
    'Rainfall: Non-stationary, non-constant mean and non-constant variance',
    'Hydrometry: Non-stationary, non-constant mean and non-constant variance',
    'Temperatur: Non-stationary variance is time-dependent (seasonality)',
    'Volume: Non-stationary, non-constant mean and non-constant variance',
    'Depth to Groundwater: Non-stationary, non-constant mean and non-constant variance',
))

cols = plotly.colors.DEFAULT_PLOTLY_COLORS

# First
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Rainfall, line=dict(color = cols[8]), name = 'basic',  legendgroup='basic'),
    row = 1,
    col = 1
)
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Rainfall.rolling(52).mean(), name = 'rolling mean', line=dict(color = cols[1]), legendgroup='mean'),
    row = 1,
    col = 1
)
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Rainfall.rolling(52).std(), name = 'rolling std', line=dict(color = cols[2]), legendgroup='std'),
    row = 1,
    col = 1
)
fig.update_yaxes(title_text='Rainfall', row=1, col=1)
fig.layout.annotations[0].update(font={'size': 9})

# Second
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Temperature, line=dict(color = cols[8]), name = 'basic', legendgroup='basic', showlegend = False),
    row = 2,
    col = 1
)
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Temperature.rolling(52).mean(), line=dict(color = cols[1]), name = 'rolling mean', legendgroup = 'mean', showlegend = False),
    row = 2,
    col = 1
)
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Temperature.rolling(52).std(), line=dict(color = cols[2]), name = 'rolling std', legendgroup = 'std', showlegend = False),
    row = 2,
    col = 1
)
fig.update_yaxes(title_text='Temperature', row=2, col=1)
fig.layout.annotations[2].update(font={'size': 9})

# Third
fig.add_trace(
    go.Scatter(x = df.Date, y = df.River_Hydrometry, line=dict(color = cols[8]), name = 'basic', legendgroup='basic', showlegend = False),
    row = 1,
    col = 2
)
fig.add_trace(
    go.Scatter(x = df.Date, y = df.River_Hydrometry.rolling(52).mean(), line=dict(color = cols[1]), name = 'rolling mean', legendgroup = 'mean', showlegend = False),
    row = 1,
    col = 2
)
fig.add_trace(
    go.Scatter(x = df.Date, y = df.River_Hydrometry.rolling(52).std(), line=dict(color = cols[2]), name = 'rolling std', legendgroup = 'std', showlegend = False),
    row = 1,
    col = 2
)
fig.update_yaxes(title_text='Hydrometry', row=1, col=2)
fig.layout.annotations[1].update(font={'size': 9})

# Fourth
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Drainage_Volume, line=dict(color = cols[8]), name = 'basic', legendgroup='basic', showlegend = False),
    row = 2,
    col = 2
)
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Drainage_Volume.rolling(52).mean(), line=dict(color = cols[1]), name = 'rolling mean', legendgroup = 'mean', showlegend = False),
    row = 2,
    col = 2
)
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Drainage_Volume.rolling(52).std(), line=dict(color = cols[2]), name = 'rolling std', legendgroup = 'std', showlegend = False),
    row = 2,
    col = 2
)
fig.update_yaxes(title_text='Volume', row=2, col=2)
fig.layout.annotations[3].update(font={'size': 9})

# Fifth
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Depth_to_Groundwater, line=dict(color = cols[8]), name = 'basic', legendgroup='basic', showlegend = False),
    row = 3,
    col = 1
)
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Depth_to_Groundwater.rolling(52).mean(), line=dict(color = cols[1]), name = 'mean', legendgroup='mean', showlegend = False),
    row = 3,
    col = 1
)
fig.add_trace(
    go.Scatter(x = df.Date, y = df.Depth_to_Groundwater.rolling(52).std(), line=dict(color = cols[2]), name = 'std', legendgroup='std', showlegend = False),
    row = 3,
    col = 1
)
fig.update_yaxes(title_text='Depth to Groundwater', row=3, col=1)
fig.layout.annotations[4].update(font={'size': 9})

fig.update_layout(height=700)

Split time series and compare the mean and variance of each partition

In [ ]:
fig = ff.create_distplot([df.Rainfall.fillna(np.inf)], ['Rainfall'], bin_size=[2])
fig.show()

In [ ]:
fig = ff.create_distplot([df.Temperature.fillna(np.inf)], ['Temperature'], bin_size=[2.5])
fig.show()

In [ ]:
fig = ff.create_distplot([df.River_Hydrometry.fillna(np.inf)], ['River_Hydrometry'], bin_size=[.1])
fig.show()

In [ ]:
fig = ff.create_distplot([df.Drainage_Volume.fillna(np.inf)], ['Drainage_Volume'], bin_size=[1000])
fig.show()

In [ ]:
fig = ff.create_distplot([df.Depth_to_Groundwater.fillna(np.inf)], ['Depth_to_Groundwater'])
fig.show()

In [ ]:
# First Order Differencing
ts_diff = np.diff(df.Depth_to_Groundwater)
df['Depth_to_Groundwater_diff_1'] = np.append([0], ts_diff)

# Second Order Differencing
ts_diff = np.diff(df.Depth_to_Groundwater_diff_1)
df['Depth_to_Groundwater_diff_2'] = np.append([0], ts_diff)

# Feature Engineering

Time Features

In [ ]:
df['year'] = pd.DatetimeIndex(df['Date']).year
df['month'] = pd.DatetimeIndex(df['Date']).month
df['day'] = pd.DatetimeIndex(df['Date']).day
df['day_of_year'] = pd.DatetimeIndex(df['Date']).dayofyear
df['week_of_year'] = pd.DatetimeIndex(df['Date']).weekofyear
df['quarter'] = pd.DatetimeIndex(df['Date']).quarter
df['season'] = df.month%12 // 3 + 1
df[['Date', 'year', 'month', 'day', 'day_of_year', 'week_of_year', 'quarter', 'season']].head()

Encoding Cyclical Features

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x = df.Date, y = df.month)
)
fig.update_yaxes(title_text='Month')
fig.update_xaxes(title_text='Date')

Such feature can be confusing for many models. We need to do something with it. Ideally, we want the underlying data to represent the same difference between two consecutive months, even between December and January. A common remedy for this issue is to encode cyclical features into two dimensions with sine and cosine transformation.

In [ ]:
month_in_year = 12
df['month_sin'] = np.sin(2*np.pi*df.month/month_in_year)
df['month_cos'] = np.cos(2*np.pi*df.month/month_in_year)

days_in_month = 30
df['day_sin'] = np.sin(2*np.pi*df.day/days_in_month)
df['day_cos'] = np.cos(2*np.pi*df.day/days_in_month)

days_in_year = 365
df['day_of_year_sin'] = np.sin(2*np.pi*df.day_of_year/days_in_year)
df['day_of_year_cos'] = np.cos(2*np.pi*df.day_of_year/days_in_year)

weeks_in_year = 52.1429
df['week_of_year_sin'] = np.sin(2*np.pi*df.week_of_year/weeks_in_year)
df['week_of_year_cos'] = np.cos(2*np.pi*df.week_of_year/weeks_in_year)

quarters_in_year = 4
df['quarter_sin'] = np.sin(2*np.pi*df.quarter/quarters_in_year)
df['quarter_cos'] = np.cos(2*np.pi*df.quarter/quarters_in_year)

seasons_in_year = 4
df['season_sin'] = np.sin(2*np.pi*df.season/seasons_in_year)
df['season_cos'] = np.cos(2*np.pi*df.season/seasons_in_year)

In [ ]:
fig = make_subplots(cols = 3, rows = 3, subplot_titles=(
    'Month',
    'Day',
    'Day of Year',
    'Week of Year',
    'Quarter',
    'Season'
))

fig.add_trace(
    go.Scatter(x = df.month_sin, y = df.month_cos),
    row = 1,
    col = 1,
)

fig.add_trace(
    go.Scatter(x = df.day_sin, y = df.day_cos),
    row = 1,
    col = 2,
)

fig.add_trace(
    go.Scatter(x = df.day_of_year_sin, y = df.day_of_year_cos),
    row = 1,
    col = 3,
)

fig.add_trace(
    go.Scatter(x = df.week_of_year_sin, y = df.week_of_year_cos),
    row = 2,
    col = 1,
)

fig.add_trace(
    go.Scatter(x = df.quarter_sin, y = df.quarter_cos),
    row = 2,
    col = 2,
)

fig.add_trace(
    go.Scatter(x = df.season_sin, y = df.season_cos),
    row = 2,
    col = 3,
)

fig.update_layout(height=700)

In [ ]:
decompose_cols = ['Rainfall', 'Temperature', 'Drainage_Volume', 'River_Hydrometry', 'Depth_to_Groundwater']

for col in decompose_cols:
    decomp = seasonal_decompose(df[col], freq=52, model='additive', extrapolate_trend='freq')
    df[f"{col}_trend"] = decomp.trend
    df[f"{col}_seasonal"] = decomp.seasonal

In [ ]:
fig = make_subplots(cols = 2, rows = 4, subplot_titles=(
    'Decomposition of Temperature',
    'Decomposition of Depth_to_Groundwater',
    '',
    '',
    '',
    ''
))

# Temperature
res = seasonal_decompose(df.Temperature, freq=52, model='additive', extrapolate_trend='freq')

fig.add_trace(
    go.Scatter(x=df.Date, y=res.observed),
    row = 1,
    col = 1
)
fig.update_yaxes(title_text='Observed', row=1, col=1)

fig.add_trace(
    go.Scatter(x=df.Date, y=res.trend),
    row = 2,
    col = 1
)
fig.update_yaxes(title_text='Trend', row=2, col=1)

fig.add_trace(
    go.Scatter(x=df.Date, y=res.seasonal),
    row = 3,
    col = 1
)
fig.update_yaxes(title_text='Seasonal', row=3, col=1)

fig.add_trace(
    go.Scatter(x=df.Date, y=res.resid),
    row = 4,
    col = 1
)
fig.update_yaxes(title_text='Residual', row=4, col=1)

# Depth to Groundwater
res = seasonal_decompose(df.Depth_to_Groundwater, freq=52, model='additive', extrapolate_trend='freq')

fig.add_trace(
    go.Scatter(x=df.Date, y=res.observed),
    row = 1,
    col = 2
)
fig.update_yaxes(title_text='Observed', row=1, col=2)

fig.add_trace(
    go.Scatter(x=df.Date, y=res.trend),
    row = 2,
    col = 2
)
fig.update_yaxes(title_text='Trend', row=2, col=2)

fig.add_trace(
    go.Scatter(x=df.Date, y=res.seasonal),
    row = 3,
    col = 2
)
fig.update_yaxes(title_text='Seasonal', row=3, col=2)

fig.add_trace(
    go.Scatter(x=df.Date, y=res.resid),
    row = 4,
    col = 2
)
fig.update_yaxes(title_text='Residual', row=4, col=2)


fig.update_layout(height=700, width=1000, showlegend=False)

# Exploratory Data Analysis

Let's begin by plotting the seasonal components of each feature and comparing the minima and maxima. 
By doing this, we can already gain some insights:

The depth to groundwater reaches its maximum around May/June and its minimum around November/December

The temperature reaches its maxmium around August and its minimum around January

The volume reaches its maximum around June and its minimum around August/September. It takes longer to reach its maximum than to reach its minimum.

The hydrometry reaches its maximum around March and its minimum around September

The volume and hydrometry reach their minimum roughly around the same time

The volume and hydrometry reach their minimum when the temperature reaches its maximum

Temperature lags begind depth to groundwater by around 2 to 3 months

In [ ]:
fig = make_subplots(rows=5, cols=1)

fig.add_trace(
    go.Scatter(x=df.Date, y=df.Depth_to_Groundwater_seasonal),
    row = 1,
    col = 1
)
fig.update_yaxes(title_text='Depth to Groundwater', row=1, col=1)
fig.update_xaxes(range=['2017-10-01', '2020-6-01'], row=1, col=1)

fig.add_trace(
    go.Scatter(x=df.Date, y=df.Temperature_seasonal),
    row = 2,
    col = 1
)
fig.update_yaxes(title_text='Temperature', row=2, col=1)
fig.update_xaxes(range=['2017-10-01', '2020-6-01'], row=2, col=1)

fig.add_trace(
    go.Scatter(x=df.Date, y=df.Drainage_Volume_seasonal),
    row = 3,
    col = 1
)
fig.update_yaxes(title_text='Drainage Volume', row=3, col=1)
fig.update_xaxes(range=['2017-10-01', '2020-6-01'], row=3, col=1)

fig.add_trace(
    go.Scatter(x=df.Date, y=df.River_Hydrometry_seasonal),
    row = 4,
    col = 1
)
fig.update_yaxes(title_text='River Hydrometry', row=4, col=1)
fig.update_xaxes(range=['2017-10-01', '2020-6-01'], row=4, col=1)

fig.add_trace(
    go.Scatter(x=df.Date, y=df.Rainfall_seasonal),
    row = 5,
    col = 1
)
fig.update_yaxes(title_text='Rainfall', row=5, col=1)
fig.update_xaxes(range=['2017-10-01', '2020-6-01'], row=5, col=1)

fig.update_layout(height=1000, title_text='Seasonal Components of Features', showlegend=False)